In [ ]:
!pip install segmentation-models
!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install 'h5py==2.10.0' --force-reinstall

In [1]:
import keras
import tensorflow as tf
assert tf.__version__ == '2.1.0', "tf version should be 2.1.0"
assert keras.__version__ == '2.3.1', "keras version should be 2.3.1"

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import json
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import cv2

from keras.models import load_model
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.optimizers import Adam

ROOT_DIR = '/content/drive/My Drive/NucleiSegmentation'
os.chdir(ROOT_DIR)

import sys
sys.path.append(os.path.join(ROOT_DIR, "src")) 
from datagen import DataGenerator
from evaluation_metrics import *
from test_utils import *

from IPython.display import clear_output
import segmentation_models as sm

Segmentation Models: using `keras` framework.


## Training

In [ ]:
# EPOCHS = 50
# EARLY_STOPPING_PATIENCE = 10
# LR_PATIENCE = 7
# BATCH_SIZE = 16
# WEIGHTS_PATH = os.path.join(ROOT_DIR,"models","weights")
# LOG_PATH = os.path.join(ROOT_DIR,"models","logs")
# MODEL = "UNet"

EPOCHS = 20
EARLY_STOPPING_PATIENCE = 10
LR_PATIENCE = 7
BATCH_SIZE = 16
WEIGHTS_PATH = os.path.join(ROOT_DIR,"models","weights")
LOG_PATH = os.path.join(ROOT_DIR,"models","logs")
MODEL = "UNet"

custom_objs = {
    'binary_crossentropy_plus_jaccard_loss': sm.losses.bce_jaccard_loss, 
    'iou_score' : sm.metrics.iou_score,
    'dice_coef' : dice_coef,
    'f1' : f1
}

for NORMALIZED_TYPE in ["unnormalized"]:
  
  for DATASET in ["MOSID", "TNBC"]:
    
    with open(os.path.join(ROOT_DIR,"data", "processed", NORMALIZED_TYPE, DATASET + ".dat"),"rb") as f : dataset = pkl.load(f)
    
    for BACKBONE in ["resnet101", "inceptionresnetv2","densenet121"]:
      
      clear_output(wait=True)
      preprocess_input =  sm.get_preprocessing(BACKBONE)
      
      X_train, y_train = dataset['Training']['TissueImages'], dataset['Training']['GroundTruth']
      X_val, y_val = dataset['Validation']['TissueImages'], dataset['Validation']['GroundTruth']
      
      X_train = preprocess_input(X_train)
      X_val = preprocess_input(X_val)

      datagen = DataGenerator(seed = 10, X_fit = X_train, y_fit = y_train)
      train_generator = datagen.get_generator(X = X_train, y = y_train, batch_size = BATCH_SIZE)
      val_generator = datagen.get_generator(X = X_val, y = y_val, batch_size = BATCH_SIZE)
      
      weight_file = "-".join([NORMALIZED_TYPE, DATASET, "UNet", BACKBONE,"best.h5"])
      model = load_model(os.path.join(ROOT_DIR, "models", "weights", weight_file), custom_objects= custom_objs, compile=False)
      model.compile(optimizer= Adam(learning_rate=0.0001) , loss = sm.losses.bce_jaccard_loss, metrics = ["accuracy", sm.metrics.iou_score, dice_coef, f1])

      
      # model = sm.Unet(BACKBONE, encoder_weights = 'imagenet')
      # model.compile(optimizer= 'adam', loss = sm.losses.bce_jaccard_loss, metrics = ["accuracy", sm.metrics.iou_score, dice_coef, f1])
      
      name_parts = [NORMALIZED_TYPE, DATASET, MODEL, BACKBONE]
      checkpoint_path = WEIGHTS_PATH + '/' + "-".join(name_parts) + "-resumed-best.h5"
      log_dir = LOG_PATH + '/' + "-".join(name_parts)

      callbacks = [
          # ReduceLROnPlateau(factor = 0.1, patience = LR_PATIENCE, monitor="val_iou_score", mode = "max", verbose = 1, min_lr = 0.00001),
          EarlyStopping(patience=EARLY_STOPPING_PATIENCE, verbose=1, monitor = "val_iou_score", mode = "max"),
          ModelCheckpoint(checkpoint_path, verbose=1, save_best_only=True, save_weights_only=False, monitor = "val_iou_score", mode = "max"),
          # TensorBoard(log_dir = log_dir , histogram_freq=1, write_images=True)
        ]
      
      # apparent work-around the callback error
      model._get_distribution_strategy = lambda: None

      print("Training ", "-".join(name_parts))
      
      train_steps, val_steps = len(X_train) / BATCH_SIZE, len(X_val) / BATCH_SIZE
      model.fit_generator(train_generator, validation_data = val_generator, epochs = EPOCHS, steps_per_epoch = train_steps, validation_steps = val_steps, callbacks = callbacks) 


## Testing

In [3]:
results = {}
for dataset in ["MOSID", "TNBC"]:
  for bool1 in [True, False]:
    for bool2 in [True, False]:
      s1 = "normalized" if bool1 else "unnormalized"
      s2 = "augmented" if bool2 else "unaugmented"
      results["-".join([s1,s2,dataset])] = get_results(dataset, pre_processing = bool1, post_processing= bool2) 


fetching results for normalized and augmented MOSID

loading normalized MOSID dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for normalized MOSID dataset are [0.3, 0.3, 0.1]
calculating individual model metrics
calculating ensembled model metrics...


performing augmented ensemble predictions:   0%|          | 0/144 [00:00<?, ?it/s]


fetching results for normalized and unaugmented MOSID

loading normalized MOSID dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for normalized MOSID dataset are [0.3, 0.3, 0.1]
calculating individual model metrics
calculating ensembled model metrics...

fetching results for unnormalized and augmented MOSID

loading unnormalized MOSID dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for unnormalized MOSID dataset are [0.3, 0.0, 0.3]
calculating individual model metrics
calculating ensembled model metrics...


performing augmented ensemble predictions:   0%|          | 0/144 [00:00<?, ?it/s]


fetching results for unnormalized and unaugmented MOSID

loading unnormalized MOSID dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for unnormalized MOSID dataset are [0.3, 0.0, 0.3]
calculating individual model metrics
calculating ensembled model metrics...

fetching results for normalized and augmented TNBC

loading normalized TNBC dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for normalized TNBC dataset are [0.3, 0.3, 0.3]
calculating individual model metrics
calculating ensembled model metrics...


performing augmented ensemble predictions:   0%|          | 0/45 [00:00<?, ?it/s]


fetching results for normalized and unaugmented TNBC

loading normalized TNBC dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for normalized TNBC dataset are [0.3, 0.3, 0.3]
calculating individual model metrics
calculating ensembled model metrics...

fetching results for unnormalized and augmented TNBC

loading unnormalized TNBC dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for unnormalized TNBC dataset are [0.1, 0.2, 0.3]
calculating individual model metrics
calculating ensembled model metrics...


performing augmented ensemble predictions:   0%|          | 0/45 [00:00<?, ?it/s]


fetching results for unnormalized and unaugmented TNBC

loading unnormalized TNBC dataset...
loading models...
making individual predictions...
fetching ensemble weights...
weights for unnormalized TNBC dataset are [0.1, 0.2, 0.3]
calculating individual model metrics
calculating ensembled model metrics...


In [ ]:
import os
import pickle as pkl
ROOT_DIR = '/content/drive/My Drive/NucleiSegmentation'
with open(os.path.join(ROOT_DIR,"models","results.dat"),"wb") as f : pkl.dump(results, f)

In [ ]:
import pickle as pkl
import os
ROOT_DIR = '/content/drive/My Drive/NucleiSegmentation'

with open(os.path.join(ROOT_DIR,"models","results.dat"),"rb") as f : results = pkl.load(f)

In [4]:
from IPython.display import display

for key in results.keys():
  data = results[key]
  df_data = {"accuracy" : [], "dice" : [], "iou" : [], "precision" : [], "recall" : [], "f1" : []}
  for model in data.keys():
    for metric in df_data.keys(): df_data[metric].append(data[model][metric])
  df = pd.DataFrame(df_data)
  df.index = data.keys()
  df = df.style.set_caption(key)
  display(df)
  print("\n")

,accuracy,dice,iou,precision,recall,f1
resnet101,0.919654,0.835548,0.808297,0.796796,0.878261,0.835548
inceptionresnetv2,0.915135,0.832051,0.802475,0.770315,0.904545,0.832051
densenet121,0.906732,0.819608,0.788005,0.744417,0.911695,0.819608
ensemble,0.925850,0.841100,0.816760,0.837802,0.844423,0.841100


,accuracy,dice,iou,precision,recall,f1
resnet101,0.914661,0.826190,0.798401,0.784362,0.872730,0.826190
inceptionresnetv2,0.911593,0.825826,0.795738,0.761644,0.901820,0.825826
densenet121,0.905143,0.816231,0.784673,0.742349,0.906446,0.816231
ensemble,0.924918,0.838697,0.814456,0.837503,0.839895,0.838697


,accuracy,dice,iou,precision,recall,f1
resnet101,0.232403,0.377154,0.116201,0.232403,1.000000,0.377154
inceptionresnetv2,0.232403,0.377154,0.116201,0.232403,1.000000,0.377154
densenet121,0.409664,0.439647,0.256755,0.282041,0.996481,0.439646
ensemble,0.643565,0.555364,0.462968,0.391053,0.957816,0.555364


,accuracy,dice,iou,precision,recall,f1
resnet101,0.232403,0.377154,0.116201,0.232403,1.000000,0.377154
inceptionresnetv2,0.232403,0.377154,0.116201,0.232403,1.000000,0.377154
densenet121,0.418192,0.440517,0.263907,0.283650,0.985561,0.440517
ensemble,0.647394,0.553612,0.465738,0.392196,0.940829,0.553612


,accuracy,dice,iou,precision,recall,f1
resnet101,0.963393,0.858534,0.855475,0.852019,0.865148,0.858533
inceptionresnetv2,0.967229,0.872809,0.868699,0.869901,0.875736,0.872808
densenet121,0.965463,0.866385,0.862688,0.860743,0.872102,0.866385
ensemble,0.966922,0.870409,0.866670,0.875709,0.865172,0.870409


,accuracy,dice,iou,precision,recall,f1
resnet101,0.960346,0.847838,0.845644,0.835609,0.860429,0.847837
inceptionresnetv2,0.967573,0.873955,0.869797,0.872357,0.875559,0.873955
densenet121,0.963897,0.859376,0.856423,0.859558,0.859193,0.859376
ensemble,0.966571,0.868772,0.865201,0.875833,0.861823,0.868772


,accuracy,dice,iou,precision,recall,f1
resnet101,0.128395,0.227572,0.064198,0.128395,1.000000,0.227571
inceptionresnetv2,0.128395,0.227572,0.064198,0.128395,1.000000,0.227571
densenet121,0.836374,0.534473,0.592053,0.421038,0.731567,0.534472
ensemble,0.861883,0.519114,0.600649,0.469392,0.580616,0.519113


,accuracy,dice,iou,precision,recall,f1
resnet101,0.128395,0.227572,0.064198,0.128395,1.000000,0.227571
inceptionresnetv2,0.128395,0.227572,0.064198,0.128395,1.000000,0.227571
densenet121,0.798887,0.487978,0.550151,0.362478,0.746401,0.487978
ensemble,0.830560,0.472165,0.562853,0.393453,0.590245,0.472164


## Plots

In [ ]:
# import matplotlib.pyplot as plt
 
# x  = ['resnet101','inceptionresnetv2','densenet121','ensemble']
# y1 = [0.703853, 0.703325, 0.689520, 0.726138]   # normalized-augmented-MOSID
# y2 = [0.703853, 0.703325, 0.689520, 0.722272]   # normalized-unaugmented-MOSID
# # y3 = [0.232403, 0.232403, 0.232403, 0.232405]   # unnormalized-augmented-MOSID
# # y4 = [0.232403, 0.232403, 0.232403, 0.232554]   # unnormalized-unaugmented-MOSID
# plt.plot(x, y1, label="normalized-augmented-MOSID")
# plt.plot(x, y2, label="normalized-unaugmented-MOSID")
# # plt.plot(x, y3, label="unnormalized-augmented-MOSID")
# # plt.plot(x, y4, label="unnormalized-unaugmented-MOSID")
# plt.plot()

# plt.xlabel("models")
# plt.ylabel("IOU score")
# plt.title("Result")
# plt.legend()
# plt.show()

# # IOU of normalized-augmented-TNBC
# import matplotlib.pyplot as plt
 
# x  = ['resnet101','inceptionresnetv2','densenet121','ensemble']
# y1 = [0.735867, 0.776128, 0.753426, 0.770552]
# y2 = [0.847837, 0.873955, 0.859376, 0.870409]
# plt.plot(x, y1, label="IOU")
# plt.plot(x, y2, label="F1")
# plt.plot()

# plt.xlabel("models")
# plt.ylabel("score")
# plt.title("Result")
# plt.legend()
# plt.show()